# What are merkle trees and how could I use them?

In this notebook I look into how merkle trees work and how I might use them.

Let me first create some batches that I want to hash. I will use the `hash` function from the `hashlib` module in Python.

In [2]:
llmBatch = [
{
id: 1,
"prompt": "Analyze sentiment of user feedback",
"model": "gpt-4-turbo",
"recipient": "0xUser1Address..."
},
{
id: 2,
"prompt": "Translate user instructions to French",
"model": "gpt-4-turbo",
"recipient": "0xUser2Address..."
},
{
id: 3,
"prompt": "Summarize the latest research on AI",
"model": "gpt-4-turbo",
"recipient": "0xUser3Address..."
},
{
id: 4,
"prompt": "Generate a creative story based on user input",
"model": "gpt-4-turbo",
"recipient": "0xUser4Address..."
}
]

Now that I have my batches, I can create a merkle tree. A merkle tree is a binary tree where each leaf node is a hash of a batch.

In [3]:
leaf_hashes = [hash(str(batch).encode('utf-8')) for batch in llmBatch]
print("Leaf hashes:", leaf_hashes)

Leaf hashes: [5536965757876449287, -8694986093143435259, -6728449945876282466, 4454228582493851044]


Now that I have the four leafs I can create the parent nodes. The parent node is the hash of the concatenation of its two child nodes.

Finally, I can create the root node, which is the hash of the concatenation of its two child nodes.

In [6]:
# create parent nodes
parent_nodes = []
for i in range(0, len(leaf_hashes), 2):
    parent_node = hash((leaf_hashes[i], leaf_hashes[i + 1]))
    parent_nodes.append(parent_node)
print("Parent nodes:", parent_nodes)

root_node = hash((parent_nodes[0], parent_nodes[1]))
print("Root node:", root_node)

Parent nodes: [1363693419216992586, -4646942462854463524]
Root node: -5862987695849600464
